In [ ]:
# jupyter-lab

!conda install -c conda-forge ipympl
# conda install nodejs -c conda-forge --repodata-fn=repodata.json
# If using JupyterLab 2
!conda install nodejs
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!jupyter lab build

# Later, if updating a previous Lab install:
!conda install ipympl
!jupyter lab build

#pip install -U jupyterlab ipywidgets jupyterlab-widgets

In [10]:
!pip install ipywidgets==7.7.2 --user

   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 9.2 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.5.2
    Uninstalling widgetsnbextension-3.5.2:
      Successfully uninstalled widgetsnbextension-3.5.2
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.6.5
    Uninstalling ipywidgets-7.6.5:
      Successfully uninstalled ipywidgets-7.6.5


In [1]:
import ipywidgets
ipywidgets.__version__

'7.7.2'

In [2]:
!pip install -U numpy
!pip install -U opencv-python
!pip install -U pandas

In [24]:
from ipywidgets import interact, fixed, interact_manual, FloatSlider, IntSlider
import math
from matplotlib import rc
rc('animation', html='jshtml')
import numpy as np
%matplotlib inline
# local modules
from util import Timer, Event, normalize_image, animate, load_events, plot_3d, event_slice

In [23]:
def high_pass_filter(event_data, cutoff_frequency=5):
    print('Reconstructing, please wait...')
    events, height, width = event_data.event_list, event_data.height, event_data.width
    events_per_frame = 2e4
    with Timer('Reconstruction'):
        time_surface = np.zeros((height, width), dtype=np.float32)
        image_state = np.zeros((height, width), dtype=np.float32)
        image_list = []
        for i, e in enumerate(events):
            beta = math.exp(-cutoff_frequency * (e.t - time_surface[e.y, e.x]))
            image_state[e.y, e.x] = beta * image_state[e.y, e.x] + e.p
            time_surface[e.y, e.x] = e.t
            if i % events_per_frame == 0:
                beta = np.exp(-cutoff_frequency * (e.t - time_surface))
                image_state *= beta
                time_surface.fill(e.t)
                image_list.append(np.copy(image_state))
    return animate(image_list, 'High Pass Filter')

def leaky_integrator(event_data, beta=1.0):
    print('Reconstructing, please wait...')
    events, height, width = event_data.event_list, event_data.height, event_data.width
    events_per_frame = 2e4
    with Timer('Reconstruction (simple)'):
        image_state = np.zeros((height, width), dtype=np.float32)
        image_list = []
        for i, e in enumerate(events):
            image_state[e.y, e.x] = beta * image_state[e.y, e.x] + e.p
            if i % events_per_frame == 0:
                image_list.append(np.copy(image_state))
    fig_title = 'Direct Integration' if beta == 1 else 'Leaky Integrator'
    return animate(image_list, fig_title)


In [22]:
with Timer('Loading'):
    n_events = 5e5
    path_to_events = 'data/boxes_6dof/events.zip'
    event_data = load_events(path_to_events, n_events)        

Loading events...
width, height: 240, 180


c:\Users\jagru\Desktop\DL\jupnote_event_demo-master\util.py:82: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  header = pd.read_csv(path_to_events, delim_whitespace=True, names=['width', 'height'],
c:\Users\jagru\Desktop\DL\jupnote_event_demo-master\util.py:86: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  event_pd = pd.read_csv(path_to_events, delim_whitespace=True, header=None,


Loaded 0.50M events
Loading: 1.82s


## Image reconstruction
Run the cell, then click ```Run Interact``` and wait for the animation to appear to see the result.  
Use the slider then click ```Run Interact``` again to try different settings.

### Leaky integrator
Equation: $L^{k+1}(x, y) = \beta L^k(x, y) + p,\ \beta \in [0, 1],\ p \in \{-1, 1\}$, where $L$ is the log image intensity and $p$ is the event polarity.

#### Set ```beta = 1``` for direct integration. Try lowering beta to see what happens.

In [ ]:
interact_manual(leaky_integrator, event_data=fixed(event_data), beta=(0, 1, 0.01));

interactive(children=(FloatSlider(value=1.0, description='beta', max=1.0, step=0.01), Button(description='Run …

### High pass filter
Below is the high pass filter as described in: https://cedric-scheerlinck.github.io/files/2018_scheerlinck_continuous-time_intensity_estimation.pdf  
Equation: $L^{k+1}(x, y) = \exp(-\alpha \Delta t) L^k(x, y) + p$, where $\alpha$ is the cutoff frequency and $\Delta t$ is the time since the last event at the same pixel.

In [ ]:
interact_manual(high_pass_filter, event_data=fixed(event_data), cutoff_frequency=(0, 20, 0.01));

interactive(children=(FloatSlider(value=5.0, description='cutoff_frequency', max=20.0, step=0.01), Button(desc…